In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import cv2
import random

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
from imblearn.over_sampling import RandomOverSampler
#from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
os.listdir('../input')

In [ ]:
path = os.path.join("../input", 'siim-isic-melanoma-classification')
print(path)

In [ ]:
os.listdir(path)

In [ ]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
train.head()

In [ ]:
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
test.head()

In [ ]:
train.index

In [ ]:
print(train.loc[train['image_name'] == 'ISIC_0068279'].index)

In [ ]:
arr =[] 
arr.append(train['target'][train.loc[train['image_name'] == 'ISIC_0068279'].index])
print(arr[0] +1)

In [ ]:
train['image_name'][0]

In [ ]:
len(train['image_name'])

In [ ]:
path = "../input/siim-isic-melanoma-classification/"
os.listdir(path)

In [ ]:
img = plt.imread(os.path.join(path, 'jpeg', 'train', train['image_name'][0] + '.jpg'))
img = cv2.resize(img, (128,128))
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
def prepare_arrays (path) :
    images_data = []
    labels_data = []
    train = pd.read_csv(os.path.join(path, 'train.csv'))
    for k in tqdm(range(len(train['image_name']))) :
        try :
            img = plt.imread(os.path.join(path, 'jpeg', 'train', train['image_name'][k] + '.jpg'))
            img = cv2.resize(img, (128,128))
            images_data.append(img)
            labels_data.append(train['target'][k])
        except :
            continue
    print("finished : ", len(images_data))
    return images_data, labels_data

In [ ]:
images_data, labels_data = prepare_arrays(path)

In [ ]:
plt.imshow(images_data[30000])
print(labels_data[30000])

In [ ]:
len(images_data)
#print(labels_data[0])

In [ ]:
images = images_data.copy()
labels = labels_data.copy()

In [ ]:
def plot_samples (images, labels) :
    real_labels = ["benign", "malignant"]
    fig = plt.figure(figsize = (20,25))
    rand = random.randint(0, len(images))

    for i in range(20): 
        fig.add_subplot(5,4, i+1)
        plt.title(real_labels[int(labels[rand])])
        plt.imshow(images[rand])
        rand = random.randint(0, len(images))

In [ ]:
plot_samples(images, labels)

# processing the data for the model

In [ ]:
def resize_array(images, size) :
    for i in range(len(images)) :
        images[i] = cv2.resize(images[i], size)
    return images

In [ ]:
le = LabelEncoder()
def preprocess_data(images, labels) :
    labels = le.fit_transform(labels)
    labels = to_categorical(labels, 2)
    images, labels = shuffle(images, labels, random_state = 32)
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.25, random_state = 32)
    x_train = np.array(x_train) 
    x_test = np.array(x_test) 
    y_train = np.array(y_train) 
    y_test = np.array(y_test)
    return x_train, x_test, y_train, y_test

In [ ]:
images = resize_array(images, tuple((32,32)))
x_train, x_test, y_train, y_test = preprocess_data(images, labels)

# modeling

In [ ]:
def build_model () :
    model = Sequential()
    model.add(Conv2D(filters = 128, kernel_size = (4,4), input_shape = (32, 32, 3), activation = 'relu'))
    model.add(MaxPool2D(pool_size = (4,4)))
    model.add(Conv2D(filters = 64, kernel_size = (2,2), activation = 'relu'))
    model.add(MaxPool2D(pool_size = (2,2)))
    model.add(BatchNormalization())
    #model.add(GlobalAveragePooling2D())

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation = 'sigmoid')) # sigmoid is better for binary classification

    model.summary()
    return model

In [ ]:
model = build_model()

In [ ]:
def train(model, epochs_num) :
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    early_stop = EarlyStopping(monitor = 'val_loss', patience = 3)
    hist = model.fit(x_train, y_train, epochs = epochs_num, validation_data = (x_test, y_test))#, callbacks = early_stop)

In [ ]:
train(model, 12)

In [ ]:
def plot_scores (model) :
    scores = pd.DataFrame(model.history.history)
    titles = ["accuaracy vs val_accuracy", "loss vs val_loss"]
    fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (24,6))
    #fig, ax = plt.figure(figsize = (20,8))

    for i in range(2): 
        fig.add_subplot(1,2, i+1)
        axes[i].set_title(titles[i])
        if i==0 : plt.plot(scores[['accuracy','val_accuracy']])#, color = 'green')
        if i==1 : plt.plot(scores[['loss','val_loss']])#, color = 'green')
        #scores[['accuracy', 'val_accuracy']].plot(ax = axes[i])

In [ ]:
plot_scores(model) 

In [ ]:
model.save('Normal_skin_cancer_model.h5')

In [ ]:
os.listdir('../input')

In [ ]:
print("final results for loss and accuracy : ")
print(model.evaluate(x_train, y_train))
print(model.evaluate(x_test,y_test))

# classification report 

In [ ]:
pred = np.argmax(model.predict(x_test), axis = -1)
y_testc = np.argmax(y_test, axis = -1)

print(classification_report(y_testc, pred))

very bad recall and f1 score class 1 (maignant)

In [ ]:
print(pred)

In [ ]:
print(y_test)

# modeling image augmentation

In [ ]:
#be calm and watch wa3i
def image_augmentation(x_train, x_test, y_train, y_test) :
    image_generator = ImageDataGenerator(
        samplewise_center = True,
        samplewise_std_normalization = True,
        rotation_range = 25, 
        width_shift_range = 0.1, 
        height_shift_range = 0.1,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True#, fill_mode = "nearst"
    )
    train_generator = image_generator.flow(x_train, y_train, batch_size = 32)
    test_generator = image_generator.flow(x_test, y_test, batch_size = 32)
    return train_generator, test_generator

In [ ]:
train_generator, test_generator = image_augmentation(x_train, x_test, y_train, y_test)

In [ ]:
modelA =  build_model()

In [ ]:
def trainA(model, epochs_num) :
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    early_stop = EarlyStopping(monitor = 'val_loss', patience = 3)
    hist = model.fit(train_generator, epochs = epochs_num, validation_data = (test_generator))#, callbacks = early_stop)

In [ ]:
trainA(modelA, 12)

In [ ]:
plot_scores(modelA) 

In [ ]:
modelA.save('Augmented_skin_cancer_model.h5')

# classification report

In [ ]:
predA = np.argmax(modelA.predict(x_test), axis = -1)
y_testc = np.argmax(y_test, axis = -1)

print(classification_report(y_testc, predA))

very bad precision and recall

# oversamping our data

In [ ]:
images_arr = np.array(images)
print(images_arr.shape)
reshaped_images = images_arr.reshape(images_arr.shape[0], -1)
print(reshaped_images.shape)

oversample = RandomOverSampler()
oversample_images_non_normal, oversample_labels = oversample.fit_resample(reshaped_images, labels)

oversample_images = oversample_images_non_normal.reshape(-1, 32,32,3)
print(oversample_images.shape)

In [ ]:
c = 0
for i in oversample_labels :
    if i == 1:
        c += 1
        
print(c)

In [ ]:
plt.imshow(oversample_images[0])

# New models

In [ ]:
x_train, x_test, y_train, y_test = preprocess_data(oversample_images, oversample_labels)
model = build_model()
train(model, 12)

In [ ]:
plot_scores(model) 

In [ ]:
pred = np.argmax(model.predict(x_test), axis = -1)
y_testc = np.argmax(y_test, axis = -1)

print(classification_report(y_testc, pred))

In [ ]:
model.save('Normal_skin_cancer_model.h5')

# oversampling with augmentation

In [ ]:
train_generator, test_generator = image_augmentation(x_train, x_test, y_train, y_test)
modelA = build_model()
trainA(modelA, 24)

In [ ]:
plot_scores(modelA) 

In [ ]:
modelA.evaluate(train_generator)
modelA.evaluate(test_generator)

In [ ]:
predA = np.argmax(modelA.predict(x_test), axis = -1)
y_testc = np.argmax(y_test, axis = -1)


print(len(x_test))
print(len(y_test))
#print(accuracy_score(y_testc, predA))
#print(classification_report(y_testc, predA))

In [ ]:
modelA.save('Augmented_skin_cancer_model.h5')

# evaluation

In [ ]:
def prepare_eval_array (path) :
    eval_array = []
    train = pd.read_csv(os.path.join(path, 'train.csv'))
    for k in tqdm(range(10)) :
        rand = random.randint(0, len(test['image_name']))
        try :
            img = plt.imread(os.path.join(path, 'jpeg', 'test', test['image_name'][rand] + '.jpg'))
            img = cv2.resize(img, (128,128))
            eval_array.append(img)
        except :
            continue
    print("finished : ", len(eval_array))
    return eval_array

In [ ]:
eval_array = prepare_eval_array(path)

In [ ]:
def plot_predictions (model) :
    fig = plt.figure(figsize = (20,8))
    real_labels = ["benign", "malignant"]

    for i in range(10) :
        img = eval_array[i]
        img = cv2.resize(img, (32,32))
        img = img.reshape(1,32,32,3)
        val = np.argmax(model.predict(img))
        fig.add_subplot(2,5, i+1)
        plt.title(real_labels[val])
        plt.imshow(eval_array[i])

In [ ]:
plot_predictions(model)

In [ ]:
plot_predictions(modelA)

# images from website

In [ ]:
os.listdir("../input")

In [ ]:
real_labels = ["benign", "malignant"]
img = plt.imread("../input/evaluation-images/binign_skin1.jpg")
img1 = img.copy()
img1 = cv2.resize(img1, (32,32))
img1 = img1.reshape(1,32,32,3)
val = np.argmax(model.predict(img1))
plt.title(real_labels[val])
plt.imshow(img)

In [ ]:
os.listdir("../input/melanoma-image")

In [ ]:
real_labels = ["benign", "malignant"]
img = plt.imread("../input/melanoma-image/melanoma-14.jpg")
img1 = img.copy()
img1 = cv2.resize(img1, (32,32))
img1 = img1.reshape(1,32,32,3)
val = np.argmax(model.predict(img1))
plt.title(real_labels[val])
plt.imshow(img)

In [ ]:
real_labels = ["benign", "malignant"]
img = plt.imread("../input/melanoma-image/melanoma-14.jpg")
img1 = img.copy()
img1 = cv2.resize(img1, (32,32))
img1 = img1.reshape(1,32,32,3)
val = np.argmax(modelA.predict(img1))
plt.title(real_labels[val])
plt.imshow(img)